### **Text Summarization on randomized data**

In [ ]:
!pip install --upgrade --no-cache-dir transformers datasets accelerate

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.5 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=d556b0b299c8b7130c6a0e508c50bc095593480ab8c5ce946fe73945fd100055
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [ ]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.7 MB/s eta 0:00:00


#### **TRAINING**

In [ ]:
# Hugging Face Transformers
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
from transformers import LEDTokenizer, LEDForConditionalGeneration


# Loading datasets
dataset = load_dataset("csv", data_files="randomized_dataset_sample.csv", sep=',', split='train')
print(dataset.column_names)

dataset = dataset.map(lambda x: {"full_note": x["full_note"], "summary": x["random_note"]})
dataset_split = dataset.train_test_split(test_size=0.2, seed=42)
train_data = dataset_split["train"]
test_data = dataset_split["test"]

# Tokenization
tokenizer = LEDTokenizer.from_pretrained("allenai/led-base-16384")

def preprocess(example):
    inputs = tokenizer(example["full_note"], truncation=True, padding="max_length", max_length=4096)
    targets = tokenizer(example["summary"], truncation=True, padding="max_length", max_length=512)
    inputs["labels"] = targets["input_ids"]
    return inputs

train_data = train_data.map(preprocess, batched=True)
test_data = test_data.map(preprocess, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

['index', 'note', 'random_note', 'sensitive_entity_note', 'full_note', 'random_full_note', 'sensitive_entity_full_note']


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [6]:
# Training Arguments
from transformers import LongformerForSequenceClassification
model_clear = LEDForConditionalGeneration.from_pretrained("allenai/led-base-16384")

# Enable gradient checkpointing on the model
model_clear.gradient_checkpointing_enable()

training_args = TrainingArguments(
    output_dir="./results_randomization_summary",        # folder for checkpoints
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,                # effectively larger batch size
    fp16=True,                                    # mixed precision training
    save_strategy="epoch",                        # save checkpoint at every epoch
    num_train_epochs=2,

    # Logging
    logging_dir="./logs",
    logging_steps=50,                             # more frequent logging
    report_to=["tensorboard","wandb"],            # log to TensorBoard and W&B

    # Learning rate and optimization
    learning_rate=2e-5,                           # more stable LR for small batches
    weight_decay=0.01,                            # regularization
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-8,
    lr_scheduler_type="linear",
    warmup_steps=100                              # warmup during first steps
)

pytorch_model.bin:   0%|          | 0.00/648M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [7]:
# Trainer
trainer = Trainer(
    model=model_clear,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
)
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: raffaeleaurucci (raffaeleaurucci-universit-degli-studi-di-salerno) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,4.344000
100,3.488200
150,3.328600
200,3.250100
250,3.193500
300,3.158500
350,3.116700
400,3.104300
450,3.090300
500,3.086400


TrainOutput(global_step=500, training_loss=3.3160644836425783, metrics={'train_runtime': 8824.9094, 'train_samples_per_second': 0.907, 'train_steps_per_second': 0.057, 'total_flos': 2.1601638678528e+16, 'train_loss': 3.3160644836425783, 'epoch': 2.0})

In [8]:
!zip -r results_randomization_summary.zip results_randomization_summary/checkpoint-500/

  adding: results_randomization_summary/checkpoint-500/ (stored 0%)
  adding: results_randomization_summary/checkpoint-500/trainer_state.json (deflated 70%)
  adding: results_randomization_summary/checkpoint-500/optimizer.pt (deflated 13%)
  adding: results_randomization_summary/checkpoint-500/config.json (deflated 61%)
  adding: results_randomization_summary/checkpoint-500/scheduler.pt (deflated 62%)
  adding: results_randomization_summary/checkpoint-500/generation_config.json (deflated 33%)
  adding: results_randomization_summary/checkpoint-500/model.safetensors (deflated 11%)
  adding: results_randomization_summary/checkpoint-500/scaler.pt (deflated 64%)
  adding: results_randomization_summary/checkpoint-500/rng_state.pth (deflated 26%)
  adding: results_randomization_summary/checkpoint-500/training_args.bin (deflated 53%)


In [ ]:
from google.colab import files

# Download file CSV
files.download("results_randomization_summary.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
losses = [x["loss"] for x in trainer.state.log_history if "loss" in x]

for step, loss in enumerate(losses, 1):
    print(f"Step {step} - Training Loss: {loss}")

Step 1 - Training Loss: 4.344
Step 2 - Training Loss: 3.4882
Step 3 - Training Loss: 3.3286
Step 4 - Training Loss: 3.2501
Step 5 - Training Loss: 3.1935
Step 6 - Training Loss: 3.1585
Step 7 - Training Loss: 3.1167
Step 8 - Training Loss: 3.1043
Step 9 - Training Loss: 3.0903
Step 10 - Training Loss: 3.0864


In [ ]:
!unzip results_encryption_summary.zip -d /content/results_encryption_summary

Archive:  results_encryption_summary.zip
   creating: /content/results_encryption_summary/results_encryption_summary/checkpoint-500/
  inflating: /content/results_encryption_summary/results_encryption_summary/checkpoint-500/trainer_state.json  
  inflating: /content/results_encryption_summary/results_encryption_summary/checkpoint-500/optimizer.pt  
  inflating: /content/results_encryption_summary/results_encryption_summary/checkpoint-500/config.json  
  inflating: /content/results_encryption_summary/results_encryption_summary/checkpoint-500/scheduler.pt  
  inflating: /content/results_encryption_summary/results_encryption_summary/checkpoint-500/generation_config.json  
  inflating: /content/results_encryption_summary/results_encryption_summary/checkpoint-500/model.safetensors  
  inflating: /content/results_encryption_summary/results_encryption_summary/checkpoint-500/scaler.pt  
  inflating: /content/results_encryption_summary/results_encryption_summary/checkpoint-500/rng_state.pth  
 

In [10]:
from transformers import pipeline, logging
from tqdm import tqdm

# Disable warnings
logging.set_verbosity_error()

# Disable gradient checkpointing
model_clear.gradient_checkpointing_disable()

# Summarization pipeline
summarizer_clear = pipeline("summarization", model=model_clear, tokenizer=tokenizer, device=0)

# Test set
test_texts = [ex["full_note"] for ex in test_data]

# List to store the generated summaries
generated_summaries = []

# Progress bar with tqdm
for text in tqdm(test_texts, desc="Generating summary"):
    summary = summarizer_clear(
        text,
        max_length=512,
        min_length=400,
        truncation=True
    )[0]["summary_text"]
    generated_summaries.append(summary)

# Number of inferences
num_inferenze = len(generated_summaries)
print(f"Number of inferences: {num_inferenze}")

# Reference summaries
references = [ex["summary"] for ex in test_data]

Generating summary:   0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/transformers/generation/utils.py:1636: UserWarning: Unfeasible length constraints: `min_length` (400) is larger than the maximum possible length (257). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(
Generating summary: 100%|██████████| 1000/1000 [57:08<00:00,  3.43s/it]


Number of inferences: 1000


In [11]:
generated_summaries[0]

'The patient was a 65-year-old aelm who presented with aaagnng of the posterior surface of the proximal third of ihsi left leg. iHs prior history included a prolonged stay in hospital for treatment of bacterial endocarditis, when five  mitral valve replacement operations were performed. iHs had also previously undergone five  abdominal operations to treat an intestinal tumor and five  varicose veins surgery.Physical examination revealed a pulsating mass in the posterior region of the proximal third of the left leg. Femoral, popliteal, and dorsal pedal pulses were palpable and normal in both lower limbs. The posterior tibial artery pulse was absent, whereas the posterior tibial artery pulse was palpable in the right lower limb.Magnetic resonance angiography showed a saccular dilatation in the tibioperoneal trunk with a 5.5 cm diameter, at the level of the origin of the posterior tibial artery. The posterior tibial artery was also occluded  . Investigation was supplemented with laborator

In [13]:
# Evaluations
import evaluate

# ROUGE
rouge = evaluate.load("rouge")

results_clear = rouge.compute(predictions=generated_summaries, references=references)
print("ROUGE - Model on randomized data:")
for k, v in results_clear.items():
    print(f"{k}: {v:.4f}")

ROUGE - Model on randomized data:
rouge1: 0.5584
rouge2: 0.4433
rougeL: 0.5244
rougeLsum: 0.5243


In [14]:
# BLEU
bleu = evaluate.load("bleu")

references_bleu = [[ref] for ref in references]

results_bleu = bleu.compute(predictions=generated_summaries, references=references_bleu)

print(f"BLEU score: {results_bleu['bleu']:.4f}")

BLEU score: 0.2675


In [15]:
from bert_score import score

# BERTScore
P, R, F1 = score(generated_summaries, references, lang="en", verbose=True)

print(f"Precision: {P.mean().item():.4f}")
print(f"Recall: {R.mean().item():.4f}")
print(f"F1: {F1.mean().item():.4f}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/32 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/16 [00:00<?, ?it/s]

done in 114.08 seconds, 8.77 sentences/sec
Precision: 0.9056
Recall: 0.8230
F1: 0.8623


In [17]:
import csv
from google.colab import files

with open("summaries_randomized.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["full_note", "summary"])
    for full_note, summary in zip(test_texts, generated_summaries):
        writer.writerow([full_note, summary])

# Download file CSV
files.download("summaries_randomized.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from statistics import mean

texts = [ex["summary"] for ex in train_data]
token_lengths = [len(tokenizer(text)["input_ids"]) for text in texts]
avg_tokens = mean(token_lengths)

print(f"Average token per entry in train set: {avg_tokens:.2f}")